In [1]:
# !pip3 install implicit

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 765 kB 11.1 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
# !wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
# !unzip ml-latest-small.zip

--2024-04-22 16:55:39--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K   695KB/s    in 1.4s    

2024-04-22 16:55:42 (695 KB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [10]:
import numpy as np
from scipy.sparse import csr_matrix

rows = []
cols = []
data = []

with open("ml-latest-small/ratings.csv", "r") as f:
    print(f.readline())

    for line in f:
        uid, mid, _, _ = line.split(",")

        rows.append(int(uid))
        cols.append(int(mid))
        data.append(1)

rows = np.array(rows)
cols = np.array(cols)
data = np.array(data)

csr = csr_matrix( (data, (rows, cols)) )

userId,movieId,rating,timestamp



In [12]:
from implicit import bpr

# factors default = 100
# 이로인해 오래걸리기 때문에 현재는 10으로 설정
model = bpr.BayesianPersonalizedRanking(factors = 10)
model.fit(csr)

  0%|          | 0/100 [00:00<?, ?it/s]

In [14]:
users = [2, 3]
ids, scores = model.recommend(users, csr[users])
print(ids) # 유저에게 추천하는 아이템의 이름
print(scores) # 내부적으로 정해진 추천 점수

[[112556  80463  69122 134130  94959 106920  68954 122904 139385  97304]
 [  1959   2792   3361   2243   3257   2352   1269   1969   2380   2496]]
[[4.008191  3.7780535 3.7551079 3.6089954 3.6076922 3.5593696 3.5360146
  3.533513  3.3664985 3.3614233]
 [2.5551362 2.46272   2.4351103 2.4225845 2.363152  2.2712119 2.1340597
  2.132045  2.129517  2.1100628]]


In [15]:
import torch

# low name and column name
users = torch.from_numpy(rows)
items = torch.from_numpy(cols)

factor = 10
item_bias = torch.randn(max(items) + 1, requires_grad=True)
item_factor = torch.randn(max(items) + 1, factor, requires_grad=True)
user_factor = torch.randn(max(users) + 1, factor, requires_grad=True)

In [34]:
optim = torch.optim.Adam([item_bias, item_factor, user_factor], lr=0.1)

logsigmoid = torch.nn.LogSigmoid()
lmd = 0.01

for epoch in range(10):
    
    negative_items = torch.randint(1, max(items) + 1, (len(users),))

    positive_preference = item_bias[items] + (user_factor[users] * item_factor[items]).sum(dim=1)
    negative_preference = item_bias[negative_items] + (user_factor[users] * item_factor[negative_items]).sum(dim=1)

    regularization = (item_bias ** 2).sum() + (item_factor ** 2).sum() + (user_factor ** 2).sum()
    cost = -logsigmoid(positive_preference - negative_items).sum() + lmd * regularization

    optim.zero_grad()
    cost.backward()
    optim.step()

    with torch.no_grad():
        train_acc = (positive_preference > negative_preference).sum() / len(positive_preference)
        print(f"epoch: {epoch}, train accuracy: {train_acc.item()}, cost: {cost.item()}")

epoch: 0, train accuracy: 0.9761989712715149, cost: 9754852352.0
epoch: 1, train accuracy: 0.9798484444618225, cost: 9760344064.0
epoch: 2, train accuracy: 0.9829723238945007, cost: 9772918784.0
epoch: 3, train accuracy: 0.9844896793365479, cost: 9725235200.0
epoch: 4, train accuracy: 0.9853723049163818, cost: 9766858752.0
epoch: 5, train accuracy: 0.9870978593826294, cost: 9756058624.0
epoch: 6, train accuracy: 0.987752377986908, cost: 9743107072.0
epoch: 7, train accuracy: 0.9879209995269775, cost: 9744765952.0
epoch: 8, train accuracy: 0.9891308546066284, cost: 9740548096.0
epoch: 9, train accuracy: 0.9891308546066284, cost: 9755918336.0


In [40]:
with torch.no_grad():
   
    outputs = 10
    uid = 2
    
    scores_all = item_bias + (user_factor[uid] * item_factor).sum(dim=1) 
    scores_all = scores_all.numpy()

    ids = np.argsort(scores_all)[::-1]
    pure_ids = ids[np.isin(ids, items[users == uid], invert=True)]
    
    pure_ids = pure_ids[:outputs]
    scores = scores_all[pure_ids]
   
    for i in range(len(pure_ids)):
        print(f"item id: {pure_ids[i]}, score: {scores[i]}")

item id: 95207, score: 85.1430892944336
item id: 99728, score: 82.00581359863281
item id: 143367, score: 76.84660339355469
item id: 59429, score: 76.25895690917969
item id: 139642, score: 75.15582275390625
item id: 103655, score: 74.18800354003906
item id: 143365, score: 71.87625885009766
item id: 184253, score: 71.51848602294922
item id: 81417, score: 71.4731216430664
item id: 110730, score: 70.53821563720703
